In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
train = pd.read_csv('../input/tabular-playground-series-sep-2021/train.csv')
train.head()

In [ ]:
sns.countplot(data = train, x = 'claim')
plt.show()

In [ ]:
def hist_plot(data):
    fig, axes = plt.subplots(20,6, figsize = (20,70))
    i= 1
    for ax in axes:
        for a in ax:
            if i <= 118:
                col = 'f{}'.format(i)
                sns.histplot(data = data, x = col, ax = a)
                a.set_title(col)
                a.set_xlabel('')
                a.set_ylabel('')
                i +=1
hist_plot(train.drop(['id', 'claim'], axis = 1))

In [ ]:
train.shape

In [ ]:
train.isna().mean()

In [ ]:
print('Unique values counts')
for col in train.columns:
    print('{}: {}'.format(col, len(train[col].unique())))

In [ ]:
train.drop(['id', 'claim'], axis = 1).describe()[1:-1].style.background_gradient(cmap ='YlOrBr', axis = None)

In [ ]:
train.drop(['id', 'claim','f35', 'f116'], axis = 1).describe()[1:-1].style.background_gradient(cmap ='YlOrBr', axis = None)


In [ ]:
pd.DataFrame(train.skew()).style.background_gradient(cmap ='YlOrBr')

In [ ]:
from sklearn.impute import KNNImputer, SimpleImputer
from sklearn.preprocessing import RobustScaler
from sklearn.pipeline import Pipeline, make_pipeline

pipeline = make_pipeline(
    (SimpleImputer(strategy='mean')),
    (RobustScaler())
)

In [ ]:
train.iloc[:,1:-1] = pipeline.fit_transform(train.drop(['id', 'claim'], axis = 1))

In [ ]:
from sklearn.model_selection import train_test_split
train_, valid_ = train_test_split(train, test_size = 0.2, stratify=train['claim'], random_state = 42)

In [ ]:
train_.shape

In [ ]:
!pip install -U lightautoml

In [ ]:
from lightautoml.automl.presets.tabular_presets import TabularAutoML
from lightautoml.tasks import Task
from lightautoml.tasks.common_metric import roc_auc_score

In [ ]:
task = Task('binary', metric= roc_auc_score)
roles = {
    'target' : 'claim',
    'drop' : 'id'
}
automl = TabularAutoML(task=task, general_params={'use_algos': [['linear_l2','cb','lgb_tuned'],['lgb', 'cb_tuned']]})

In [ ]:
oof_pred = automl.fit_predict(train_, roles= roles)

In [ ]:
valid_pred = automl.predict(valid_)

In [ ]:
roc_auc_score(valid_['claim'], valid_pred.data[:, 0])

In [ ]:
oof_pred = automl.fit_predict(train, roles= roles)

In [ ]:
test = pd.read_csv('../input/tabular-playground-series-sep-2021/test.csv')
test

In [ ]:
test.iloc[:, 1:] = pipeline.transform(test.drop('id', axis = 1))

In [ ]:
test_pred = automl.predict(test)

In [ ]:
submiss = pd.read_csv('../input/tabular-playground-series-sep-2021/sample_solution.csv', index_col=0)
submiss['claim'] = test_pred.data
submiss.to_csv('./my_submissv2.csv')

Upvote for me, thank you!!!